In [1]:
import datetime
from pynq import Overlay
from pynq import DefaultIP
from pynq import DefaultHierarchy
from pynq import Xlnk
from pynq import MMIO
from pynq.pl import *
import pynq.lib.dma
import numpy as np
import time

In [2]:
XMUL_MATRIX_AXILITES_ADDR_A_DATA = 0x10
XMUL_MATRIX_AXILITES_BITS_A_DATA = 32
XMUL_MATRIX_AXILITES_ADDR_B_DATA = 0x18
XMUL_MATRIX_AXILITES_BITS_B_DATA = 32
XMUL_MATRIX_AXILITES_ADDR_C_DATA = 0x20
XMUL_MATRIX_AXILITES_BITS_C_DATA = 32

def init_hw(filepath):
    global ol, xlnk, ip
    xlnk = Xlnk()
    ol = Overlay(filepath)
    ip = ol.mul_matrix_0

In [3]:
init_hw("/home/xilinx/jupyter_notebooks/mklabs/design_1.bit")
ol?

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


In [4]:
DIM = 2048

a = np.ones((DIM,DIM)).astype('int') * 3
b = np.ones((DIM,DIM)).astype('int') * 3
c = np.zeros((DIM,DIM)).astype('int')

In [7]:
xlnk.xlnk_reset()
ip.write(0x00, 4)
ip.read(0x00)

4

In [8]:
%%time

xlnk.xlnk_reset()
ip.write(0x00, 4)
fpga_state = ip.read(0x00)

a_p = xlnk.cma_array(shape=((DIM, DIM)), dtype=np.int32)
b_p = xlnk.cma_array(shape=((DIM, DIM)), dtype=np.int32)
c_p = xlnk.cma_array(shape=((DIM, DIM)), dtype=np.int32)

np.copyto(a_p, a)
np.copyto(b_p, b)
np.copyto(c_p, c)

a_p_ptr = a_p.physical_address
b_p_ptr = b_p.physical_address
c_p_ptr = c_p.physical_address

ip.write(0x00, 4)

if fpga_state == 4:
    ip.write(XMUL_MATRIX_AXILITES_ADDR_A_DATA, a_p_ptr)
    ip.write(XMUL_MATRIX_AXILITES_ADDR_B_DATA, b_p_ptr)
    ip.write(XMUL_MATRIX_AXILITES_ADDR_C_DATA, c_p_ptr)
else:
    print("Can't write values, must be in IDLE state")
    raise KeyboardInterrupt
    
ip.write(0x00, 1)
fpga_state = ip.read(0x00)

max_try = 1000000
while fpga_state != 6 and fpga_state != 4:
    fpga_state = ip.read(0x00)
    max_try = max_try -1
    if max_try == 0:
        print("ERROR: Can't go ahead")
        ip.write(0x00, 4)
        raise KeyboardInterrupt
        
ip.write(0x00, 4)
c[:, :] = c_p

CPU times: user 3.42 s, sys: 150 ms, total: 3.57 s
Wall time: 3.57 s
